In [11]:
pip install tensorflow==2.5.0

     -------------------------------------- 422.6/422.6 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 13.3/13.3 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 132.0/132.0 kB 2.6 MB/s eta 0:00:00



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\tayal\\anaconda3\\Lib\\site-packages\\~~mpy\\.libs\\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
     ---------------------------------------- 2.7/2.7 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 462.4/462.4 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 2.9/2.9 MB 4.1 MB/s eta 0:00:00
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=4a4ed4f2a8a0de097243486f3e929cf22e1a186d40024be93dadd95f3ca20101
  Stored in directory: c:\users\tayal\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting u

In [12]:
import tensorflow as tf
slim = tf.contrib.slim
import sys
sys.path.append('/Users/marcotcr/phd/tf-models/slim')
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [ ]:
from nets import inception
from preprocessing import inception_preprocessing
session = tf.Session()
image_size = inception.inception_v3.default_image_size
def transform_img_fn(path_list):
    out = []
    for f in path_list:
        image_raw = tf.image.decode_jpeg(open(f).read(), channels=3)
        image = inception_preprocessing.preprocess_image(image_raw, image_size, image_size, is_training=False)
        out.append(image)
    return session.run([out])[0]
from datasets import imagenet
names = imagenet.create_readable_names_for_imagenet_labels()
processed_images = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))
import os
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, _ = inception.inception_v3(processed_images, num_classes=1001, is_training=False)
probabilities = tf.nn.softmax(logits)

checkpoints_dir = '/Users/marcotcr/phd/tf-models/slim/pretrained'
init_fn = slim.assign_from_checkpoint_fn(
    os.path.join(checkpoints_dir, 'inception_v3.ckpt'),
    slim.get_model_variables('InceptionV3'))
init_fn(session)
def predict_fn(images):
    return session.run(probabilities, feed_dict={processed_images: images})
